# Class that implements Emission Matrix for IBD detection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import socket as socket
import os as os
import sys as sys
import multiprocessing as mp
import h5py
import allel
import itertools as it

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM O2 Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/hapBLOCK/"  # The Path on Harvard Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)

print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

compute-a-16-54.o2.rc.hms.harvard.edu
HSM O2 Computational partition detected.
/n/groups/reich/hringbauer/git/hapBLOCK
CPU Count: 32


In [17]:
"""
Class containing functions to calculate Emission Probabilities.
Contains Sub-Classes, as well as factory Method to return the 
correct subclass
@ Author: Harald Ringbauer, September 2020
"""

###############################
###############################

class Emissions(object):
    """Class for emission probabilities
    Has methods to return emission probabilities"""

    def give_emission_matrix(self, **kwargs):
        """Return Emission Matrix - for every possible set of states"""
        raise NotImplementedError("Implement This in specific subclass.")
    
    def give_emission_log(self, **kwargs):
        """Return the full emission Probability directly in Log Space. 
        By default just log the emissiom matrix. Can be overwritten for
        nice computational properties"""
        m = self.give_emission_matrix(**kwargs)
        #assert(np.min(m)>0)
        return np.log(m)

    def set_params(self, **kwargs):
        """Set the Parameters.
        Takes keyworded arguments"""
        for key, value in kwargs.items():
            setattr(self, key, value)
            
class HaplotypeSharingEmissions(Emissions):
    """Class for emission probabilities of diploid haplotypes who
    possibly share one haplotype"""
    
    def hw_prob_haplo_pp(self, ht_p, p):
        """Calculate HW Probabilitiy of haplotype gt
        ht_p: [l,2] Array of haplotype likelihood, l locis, 2: Nr of genotypes.
        First one is ancestral. Second one is derived prob.
        p: [l] Array of (derived) allele frequencies
        returns [l] vector of HW prob of gt"""
        prob = ht_p[:,1] * p +  ht_p[:,0] * (1-p)
        return prob

    def hw_prob_haplo_share_pp(self, ht_p1, ht_p2, p):
        """Calculate probability of sharing haplotypes
        ht_p1, ht_p2: Array of haplotype likelihood, l locis, 2: Nr of genotypes.
        p: [l] Array of (derived) allele frequencies
        returns [l] vector of prob that shared haplotype"""
        p_hw = np.stack((1-p,p), axis=1)
        prob = np.sum(ht_p1 * ht_p2 * p_hw, axis=1)
        return prob

    def hw_prob_haplos_pp(self, hts_p, p):
        """Calculate HW Probabilitiy of haplotype gt
        gt: [k,l,2] Array of haplotype likelihood, k: Nr of haplotypesl locis, 2: Nr of genotypes.
        First one is ancestral. Second one is derived prob.
        p: [l] Array of (derived) allele frequencies
        returns [l] vector of HW prob of gt"""
        prob = hts_p[:,:,1] * p +  hts_p[:,:,0] * (1-p)
        prob_tot = np.prod(prob, axis=0)
        return prob_tot

    def hw_probs_shared(self, hts_p, p, shared=(0,2), dtype="float"):
        """Give emission probabilities for shared state.
        Assume 0/1 2/3 are diploid haplotypes, and 0/2 are shared
        hts_p: [4,l,2] Array of four haplotype probabilities
        p: [l] array of derived genotype probability.
        shared: tuple of length 2 giving the indices of the shared haplotypes"""
        not_shared = [i for i in range(0,4) if i not in shared]
        assert(len(not_shared)==2 & len(shared)==2)
        p_hw1 = self.hw_prob_haplo_pp(hts_p[not_shared[0],:,:], p=p)
        p_hw2 = self.hw_prob_haplo_pp(hts_p[not_shared[1],:,:], p=p)
        p_shared = self.hw_prob_haplo_share_pp(hts_p[shared[0],:,:],hts_p[shared[1],:,:], p=p)
        p = p_hw1*p_hw2*p_shared
        return p

    def give_emission_matrix(self, hts_p, p, dtype="float"):
        """Give emission Matrix for 5-state HMM.
        0th state: HW 1st-4th State: Haplotype Copying
        Input: p: [l] Array of (derived) allele frequencies
        hts_p: [4,l,2] Array of four haplotype probabilities.
        Return: emission matrix [5,l]."""
        l = np.shape(hts_p)[1]
        e_mat = np.zeros((5,l), dtype=dtype)
        e_mat[0,:] = self.hw_prob_haplos_pp(hts_p,p=p)
        for i, (j,k) in enumerate(it.product([0,1],repeat=2)):
            e_mat[i+1,:] = self.hw_probs_shared(hts_p, p=p, shared=[j,k+2])
        return e_mat
    
def load_emission_model(e_model="haploid_gl"):
    """Factory method to return the right Emission Model"""
    if e_model == "haploid_gl":
        e_obj = HaplotypeSharingEmissions()
    else:
        raise NotImplementedError("Emission Model not found!")
    return e_obj

In [33]:
0.9*0.8*0.7

0.504

In [97]:
hw_prob_haplo_share_pp(hts[0,:,:], hts[1,:,:],p=p)

array([0.9 , 0.2 , 0.  , 0.25])

In [98]:
hw_probs_shared(hts2,p=p)

array([0.729 , 0.008 , 0.    , 0.0625])

In [100]:
hw_probs_shared(hts2, p=p, shared=(1,2))

array([0.729 , 0.008 , 0.063 , 0.0625])

In [43]:
0.2**4

0.0016000000000000003

In [ ]:
give_emission_matrix(ht2, p)

In [6]:
e = load_emission_model(e_model="haploid_gl")

In [16]:
e.hw_prob_haplos_pp(hts2, p)

array([0.6561, 0.0016, 0.0189, 0.0625])

In [13]:
e.give_emission_matrix(hts2, p)

array([[0.6561, 0.0016, 0.0189, 0.0625],
       [0.729 , 0.008 , 0.    , 0.0625],
       [0.729 , 0.008 , 0.    , 0.0625],
       [0.729 , 0.008 , 0.063 , 0.0625],
       [0.729 , 0.008 , 0.063 , 0.0625]])

### IBD State
w.l.o.g. assume first allele shared, second allele not shared

In [11]:
### Input:
# gt1, gt2: Genotype vectors: [l,2] 0: anc, 1 derived alle
# pl1, pl2: genotype probabilities: [l]
# p: allele frequencies: [l]

In [13]:
### Probs for alternative alleles:
b1 = gt1[:,1] * p + (1-gt1[:,1]) * (1-p) 
b2 = gt2[:,1] * p + (1-gt2[:,1]) * (1-p)

### Prob identical
b3 = gt1[:,0] * gt2[:,0] * (1-p) + \  # prob homo anc
     (1-gt1[:,0]) * (1-gt2[:,0]) * p  # prob homo derived
    
b = b1 * b2 * b3

### Do the four sharing states

In [46]:
(2*0.9*0.1)**2

0.032400000000000005

# Test data

In [3]:
gt1 = np.array([[0,1,0],[0,0,1],[1,0,0],[0,1,0]]).T
gt2 = np.array([[0,1,0],[0,0,1],[0,0,1],[0,0,1]]).T
gts = np.stack((gt1, gt2))
p = np.array([0.9,0.8,0.7,0.5])

In [7]:
### Test data for biallelic case
ht1 = np.array([[0,1],[1,0],[0,1],[0.1,0.9]])
ht2 = np.array([[0,1],[1,0],[1,0],[0.5,0.5]])
ht3 = np.array([[0,1],[1,0],[1,0],[0.5,0.5]])
ht4 = np.array([[0,1],[1,0],[1,0],[0.5,0.5]])
hts = np.stack((ht1, ht2))
hts2 = np.stack((ht1,ht2,ht3,ht4))
p = np.array([0.9,0.8,0.7,0.5])

In [12]:
np.shape(hts2)

(4, 4, 2)

In [34]:
a=np.stack((1-p,p), axis=1)

In [44]:
0.3**3 * 0.7

0.018899999999999997

In [20]:
p

array([0.9, 0.8, 0.7, 0.5])

In [23]:
0.9 * 0.2 * 0.3 * 0.5

0.027000000000000003

In [27]:
0.1 * 0.2 *0.3*0.5

0.0030000000000000005

In [26]:
p

array([0.9, 0.8, 0.7, 0.5])